## Langchain
### Agent

In [ ]:
# Static Model
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-5",
    temperature=0.1,
    max_tokens=1000,
    timeout=30
    # ... (other params)
)
agent = create_agent(model)

# model = ChatOpenAI(
#     model="ai/phi4",
#     temperature=0.1,
#     max_tokens=1000,
#     timeout=30,
#     base_url="http://localhost:12434/engines/v1",
#     api_key="docker",
# )
# agent = create_agent(model, tools=tools)

In [6]:
# Dynamic Model
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest #, ModelResponse



@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}"

@tool
def get_weather(location: str) -> str:
    """Get weather information for a location."""
    return f"Weather in {location}: Sunny, 72°F"

tools = [search, get_weather]

basic_model = ChatOpenAI(model="ai/gemma3", base_url="http://localhost:12434/engines/v1", api_key="docker")
advanced_model = ChatOpenAI(model="ai/phi4", base_url="http://localhost:12434/engines/v1", api_key="docker")


@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler):# -> ModelResponse:
    """Choose model based on conversation complexity."""
    message_count = len(request.state["messages"])

    if message_count > 10:
        # Use advanced model for longer conversations
        model = advanced_model
    else:
        model = basic_model

    request.model = model
    return handler(request)


agent = create_agent(
    model=basic_model,  # Default model
    tools=tools,
    middleware=[dynamic_model_selection],
    system_prompt="You are a helpful assistant. Be concise and accurate."
)

agent.invoke( {"messages": [{"role": "user", "content": "what is the weather in sf"}]})

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='bd3680f4-6663-4afa-9299-099614c895d0'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 342, 'total_tokens': 385, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ai/gemma3', 'system_fingerprint': 'b1-ca71fb9', 'id': 'chatcmpl-raSBLCN6YDtqxtek2gOeWd99tHkzdXi2', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--fe21f453-8442-407c-93af-696dfd4000dc-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'sf'}, 'id': 'iaQzFbhfhP3GeKbxeeg6JolNGiGKlsir', 'type': 'tool_call'}], usage_metadata={'input_tokens': 342, 'output_tokens': 43, 'total_tokens': 385, 'input_token_details': {}, 'output_token_details': {}}),
  ToolMessage(content='Weather in sf: Sunny, 72°F', name='get_weather', id='2fb369e8-

In [8]:
# Dynamic System Prompt
from typing import TypedDict

from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest

class Context(TypedDict):
    user_role: str

@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on user role."""
    user_role = request.runtime.context.get("user_role", "user")
    base_prompt = "You are a helpful assistant."

    if user_role == "expert":
        return f"{base_prompt} Provide detailed technical responses."
    elif user_role == "beginner":
        return f"{base_prompt} Explain concepts simply and avoid jargon."

    return base_prompt

agent = create_agent(
    basic_model,
    tools=[],
    middleware=[user_role_prompt],
    context_schema=Context
)

# The system prompt will be set dynamically based on context
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Explain machine learning"}]},
    context={"user_role": "expert"}
)

In [9]:
result

{'messages': [HumanMessage(content='Explain machine learning', additional_kwargs={}, response_metadata={}, id='d83febd1-674c-4df7-a6f5-f7f850b2e52f'),
  AIMessage(content='Okay, let\'s dive into machine learning. It’s a massive and rapidly evolving field, so I’ll break it down into manageable pieces, covering the core concepts, different types, and some key considerations.\n\n**1. What is Machine Learning? (The Basic Idea)**\n\nAt its core, machine learning is about enabling computers to *learn* from data without being explicitly programmed.  Traditional programming involves writing specific instructions for a computer to follow. Machine learning, however, focuses on feeding a computer data and letting it figure out the rules and patterns itself.\n\nThink of it like teaching a dog a trick. You don’t tell the dog *exactly* how to sit (e.g., "move your rear end 30 degrees, bend your legs at 45 degrees..."). Instead, you show the dog what "sit" looks like, give it a reward when it does it

In [10]:
# Structured output
# ToolStrategy

from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str

agent = create_agent(
    basic_model,
    tools=[],
    response_format=ToolStrategy(ContactInfo)
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

In [11]:
# ProviderStrategy
from langchain.agents.structured_output import ProviderStrategy

agent = create_agent(
    basic_model,
    response_format=ProviderStrategy(ContactInfo)
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result

{'messages': [HumanMessage(content='Extract contact info from: John Doe, john@example.com, (555) 123-4567', additional_kwargs={}, response_metadata={}, id='02819f7c-1969-4954-9865-6916eb999c3b'),
  AIMessage(content='{\n  "name": "John Doe",\n  "email": "john@example.com",\n  "phone": "(555) 123-4567"\n}\n', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 37, 'total_tokens': 83, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ai/gemma3', 'system_fingerprint': 'b1-ca71fb9', 'id': 'chatcmpl-ZeN46G2kl6NkQ2enbsk2XKQPSKP1HQeK', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--d8f6d65c-f6bc-4cdd-8f9b-aee8e928ce8f-0', usage_metadata={'input_tokens': 37, 'output_tokens': 46, 'total_tokens': 83, 'input_token_details': {}, 'output_token_details': {}})],
 'structured_response': ContactInfo(name='John Doe', email='john@example.

In [12]:
# Memory

from typing import Annotated, TypedDict
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import AgentMiddleware

# Define custom state extending AgentState
class CustomAgentState(AgentState):
    """Extended state with user preferences."""
    user_preferences: dict

# Create middleware with custom state
class PreferencesMiddleware(AgentMiddleware[CustomAgentState]):
    state_schema = CustomAgentState  # Set the state schema

agent = create_agent(
    basic_model,
    tools=tools,
    middleware=[PreferencesMiddleware()]
)

# The agent can now track additional state beyond messages
result = agent.invoke({
    "messages": [{"role": "user", "content": "I prefer technical explanations"}],
    "user_preferences": {"style": "technical", "verbosity": "detailed"},
})

result

{'messages': [HumanMessage(content='I prefer technical explanations', additional_kwargs={}, response_metadata={}, id='779c8537-f5a1-4c9a-a04b-06c395e4adcb'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 340, 'total_tokens': 387, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ai/gemma3', 'system_fingerprint': 'b1-ca71fb9', 'id': 'chatcmpl-wbcPF6Ep4qxVwwxgEbes1XXqmsH3dt14', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--036f915d-acbd-4c63-b85f-3929b6bc085a-0', tool_calls=[{'name': 'search', 'args': {'query': 'What is the current weather in London?'}, 'id': 'fncHftrKxe9TKjxqmYgVeVXYtH2weR1Q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 340, 'output_tokens': 47, 'total_tokens': 387, 'input_token_details': {}, 'output_token_details': {}}),
  ToolMessage(content='Results for: What is the cur

In [ ]:
# Before model hook
from langchain.messages import RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import before_model
from langgraph.runtime import Runtime

@before_model
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Keep only the last few messages to fit context window."""
    messages = state["messages"]

    if len(messages) <= 3:
        return None  # No changes needed

    first_msg = messages[0]
    recent_messages = messages[-3:] if len(messages) % 2 == 0 else messages[-4:]
    new_messages = [first_msg] + recent_messages

    return {
        "messages": [
            RemoveMessage(id=REMOVE_ALL_MESSAGES),
            *new_messages
        ]
    }

agent = create_agent(
    advanced_model,
    tools=tools,
    middleware=[trim_messages]
)

In [ ]:
# After model hook
from typing import Any
from langchain.messages import AIMessage, RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import after_model
from langgraph.runtime import Runtime

@after_model
def validate_response(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Check model response for policy violations."""
    messages = state["messages"]
    last_message = messages[-1]

    if "confidential" in last_message.content.lower():
        return {
            "messages": [
                RemoveMessage(id=REMOVE_ALL_MESSAGES),
                *messages[:-1],
                AIMessage(content="I cannot share confidential information.")
            ]
        }

    return None  # No changes needed

agent = create_agent(
    model,
    tools=tools,
    middleware=[validate_response]
)

In [13]:
# Streaming
for chunk in agent.stream({
    "messages": [{"role": "user", "content": "Search for AI news and summarize the findings"}]
}, stream_mode="values"):
    # Each chunk contains the full state at that point
    latest_message = chunk["messages"][-1]
    if latest_message.content:
        print(f"Agent: {latest_message.content}")
    elif latest_message.tool_calls:
        print(f"Calling tools: {[tc['name'] for tc in latest_message.tool_calls]}")

Agent: Search for AI news and summarize the findings
Calling tools: ['search']
Agent: Results for: AI news summary
Agent: Here's a summary of recent AI news:

*   **Google's Gemini AI Model Faces Scrutiny:** Concerns are rising regarding Gemini's performance and potential biases. Reports suggest it struggles with certain reasoning tasks and exhibits problematic behavior in some tests.
*   **OpenAI’s GPT-4o Launch:** OpenAI has released GPT-4o, a multimodal model capable of processing audio, text, and images in real-time. It’s being lauded for its speed and responsiveness in conversational settings.
*   **AI Regulation Discussions Continue:** Governments worldwide are intensifying discussions on regulating AI development and deployment. The EU's AI Act is nearing finalization, and the US is exploring various approaches.
*   **AI-Powered Drug Discovery Advances:** Several companies are utilizing AI to accelerate drug discovery, with promising results in identifying potential drug candida